In [1]:
import mysql.connector
import json
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# Load database configuration from a JSON file
with open("config.json", "r") as f:
    config = json.load(f)

In [2]:
# Create a database connection
cnx = mysql.connector.connect(
    host="localhost",  # Update with your host
    user=config["username"],
    password=config["password"],
    database="project"
)
# Create a cursor object to execute SQL queries
cursor = cnx.cursor()


In [3]:
# Execute the first query and fetch the results
query1 = "SELECT Name, Location FROM Organization;"
cursor.execute(query1)
results1 = cursor.fetchall()
df=pd.DataFrame(results1,columns=["ORG","Location"])
print(df.head())

    ORG   Location
0  Org1  Location1
1  Org2  Location2
2  Org3  Location3
3  Org4  Location4
4  Org5  Location5


In [4]:
query2 = "SELECT CommentID,Content,UserID,PostID FROM Comment;"
cursor.execute(query2)
results2 = cursor.fetchall()
df=pd.DataFrame(results2,columns=["CommentID","Content","UserID","PostID"])
for i in range(len(df)):
    c_id=df["CommentID"][i]
    scores=sid.polarity_scores(df["Content"][i])
    print(f"{c_id}:{scores}")


1:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
2:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
3:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
4:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
5:{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
6:{'neg': 0.513, 'neu': 0.487, 'pos': 0.0, 'compound': -0.2755}


In [7]:
query2 = "SELECT * FROM User;"
cursor.execute(query2)
results2 = cursor.fetchall()
print(results2)

[(1, 'User1', 'user1@example.com', 'voter', 'password1', None), (2, 'User2', 'user2@example.com', 'voter', 'password2', None), (3, 'User3', 'user3@example.com', 'representative', 'password3', 1), (4, 'User4', 'user4@example.com', 'voter', 'password4', None), (5, 'User5', 'user5@example.com', 'representative', 'password5', 2)]


In [12]:
def get_user_by_email(email):
    query = f"SELECT * FROM User WHERE Email = '{email}'"
    cursor.execute(query)
    result = cursor.fetchall()
    return result

In [15]:
len(get_user_by_email("user1@example.com"))

1

In [16]:
len(get_user_by_email("atharva@example.com"))

0

In [23]:
user=get_user_by_email("user1@example.com")
user[0]

(1, 'User1', 'user1@example.com', 'voter', 'password1', None)

In [5]:
query="SELECT OrganizationID FROM Organization where Name='Org2'"
cursor.execute(query)
result = cursor.fetchall()
result[0][0]

2

In [8]:
query = f"SELECT * FROM User"
cursor.execute(query)
result = cursor.fetchall()
posts_df = pd.DataFrame(result, columns=["UserID","Name","Email","Role","Password","OrganizationID","profile_pic"])

In [10]:
posts_df

,UserID,Name,Email,Role,Password,OrganizationID,profile_pic
0,1,amaan,amaan@coep.com,representative,$2b$12$dEn9s1j.NqOx89M5JqMg9OMq1QdqTR/xXNaVPP4...,NaN,None
1,2,thakre,thakre@thakre.com,voter,$2b$12$9XtcJazxqboB1V3x9L0uze/NhDx/SY2un6nDJj2...,NaN,None
2,3,atharvaam21,atharvaam21@gmail.com,representative,$2b$12$FeFm/cTdpQk2DxuuuFtC3.VPpJ60zthP3whYNf6...,3.0,None
3,4,pratyay,pratyay@gmail.com,representative,$2b$12$Y6Q4/vIOqCELDGUX/Tq87OXA7T8C0TIFoA/pUZq...,1.0,None
4,5,chou,chou@thetou.com,representative,$2b$12$eyGqnw0Lmc.y/ID/aFwX0OxNffs7bEEHopLs.Ba...,3.0,None
5,6,user1,user1@example.com,voter,$2b$12$wNQnFUBz6w8Ek30n7nWUJ./h8G1gP3KXUBWfQrM...,3.0,b'static/profile_images\\Screenshot_2023-10-21...


In [17]:
posts_df=posts_df[posts_df['UserID']==6]
(posts_df['profile_pic'].values)[0].decode('utf-8').replace('\\','/')

'static/profile_images/Screenshot_2023-10-21_155945.png'

In [9]:
query = f"SELECT * FROM Post"
cursor.execute(query)
result = cursor.fetchall()
result

[(1,
  'Jharkhand water crisis',
  '70% Dams have dried up in this summer. Requesting AID immediately',
  datetime.datetime(2023, 11, 8, 7, 57, 21),
  'in progress',
  'environment',
  1,
  4,
  2,
  None,
  'issue'),
 (2,
  'Pune Road Crisis in Baner',
  'Traffic on average 2hrs per day due to poor road infra. ',
  datetime.datetime(2023, 11, 8, 8, 2, 40),
  'in progress',
  'governance',
  2,
  2,
  3,
  None,
  'issue'),
 (3,
  'Afforestation act of 2023- Maharashtra Govt',
  'Planning on increasing area under tree foliage by 25% till 2030',
  datetime.datetime(2023, 11, 8, 8, 6, 59),
  'in progress',
  'environment',
  3,
  4,
  0,
  3,
  'project'),
 (4,
  'Maharashtra Temperature Reach High 20s in Peak Winter',
  'This unexpected rise in temperatures has left many residents bewildered as they navigate through days that feel more like early spring than peak winter. For a region accustomed to cozy sweaters and hot beverages, the sight of people strolling around in light attire and 

In [7]:
def get_posts():
    query = f"SELECT Post.*,Name FROM Post join User using(UserID);"
    cursor.execute(query)
    result = cursor.fetchall()
    posts_df=pd.DataFrame(result,columns=["PostID","Title","Description","CreatedAtDate","Status","Topic","UserID","Upvotes","Downvotes","OrganizationID","Type","PostedBy"])
    return posts_df

In [8]:
get_posts()

,PostID,Title,Description,CreatedAtDate,Status,Topic,UserID,Upvotes,Downvotes,OrganizationID,Type,PostedBy
0,1,Post1,Description1,2023-11-02 09:26:24,in progress,environment,1,13,3,NaN,issue,User1
1,2,Post2,Description2,2023-11-02 09:26:24,in progress,social,2,16,5,NaN,issue,User2
2,3,Post3,Description3,2023-11-02 09:26:24,in progress,environment,3,6,2,1.0,issue,User3
3,4,Post4,Description4,2023-11-02 09:26:24,in progress,governance,4,8,1,NaN,issue,User4
4,5,Post5,Description5,2023-11-02 09:26:24,in progress,environment,5,12,5,2.0,issue,User5
5,6,tHIS is a Title (i am drunk),Description1,2023-11-02 09:31:35,in progress,environment,3,11,2,NaN,issue,User3
6,7,Jharkhand water crisis,81% dams dried up. requesting AID immediately,2023-11-04 07:39:29,in progress,environment,8,1,1,3.0,issue,amaan
7,8,Pune Road Crisis in Baner,"Potholes, Traffic and lack of management",2023-11-04 08:17:16,in progress,governance,8,0,1,3.0,issue,amaan
8,9,Afforestation act of 2023- Maharashtra Govt,To increase land under tree foliage by 15% by ...,2023-11-04 10:20:55,in progress,environment,8,1,0,3.0,project,amaan
9,10,The Anti LGBTQ Law of 2023-Forever,Being part of the LGBTQ is Gay,2023-11-04 10:25:43,in progress,social,10,3,0,3.0,project,chou


# Handling Voting

In [3]:
def has_user_upvoted(user_id, post_id):
    # Check if the user has upvoted the post
    query = f"SELECT VoteType FROM UserVotes WHERE UserID = {user_id} AND PostID = {post_id}"
    cursor.execute(query)
    result = cursor.fetchall()

    if len(result) > 0:
        votetype=result[0][0]
        if votetype=='downvote':
            # User has upvoted, handle downvoting logic
            # First, remove the previous upvote from the votes table
            delete_query = f"DELETE FROM UserVotes WHERE UserID = {user_id} AND PostID = {post_id};"
            cursor.execute(delete_query)
            cnx.commit()

            # Next, update the upvotes and downvotes in the posts table
            update_query = f"UPDATE Post SET Downvotes = Downvotes - 1 WHERE PostID = {post_id}"
            cursor.execute(update_query)
            cnx.commit()
            return False  # User has downvoted after the previous upvote
        else:
            return True
    else:
        return False  # User has not upvoted the post, they can upvote it


In [5]:
def insert_upvote(user_id, post_id):
    query = f"INSERT INTO UserVotes (UserID, PostID, VoteType) VALUES ({user_id}, {post_id}, 'upvote')"
    cursor.execute(query)
    cnx.commit()
    update_query = f"UPDATE Post SET Upvotes = Upvotes + 1 WHERE PostID = {post_id}"
    cursor.execute(update_query)
    cnx.commit()


In [9]:
if not has_user_upvoted(1,1):
    insert_upvote(1,1)

In [10]:
query="SELECT UserVotes.*,Upvotes,Downvotes FROM UserVotes join Post using(PostID)"
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(1, 1, 1, 'upvote', datetime.datetime(2023, 11, 4, 6, 6, 15), 11, 2)]


In [ ]:
def has_user_downvoted(user_id, post_id):
    # Check if the user has upvoted the post
    query = f"SELECT VoteType FROM UserVotes WHERE UserID = {user_id} AND PostID = {post_id} AND VoteType = 'upvote'"
    cursor.execute(query)
    result = cursor.fetchall()

    if len(result) > 0:
        votetype=result[0][0]
        if votetype=='downvote':
            # User has upvoted, handle downvoting logic
            # First, remove the previous upvote from the votes table
            delete_query = f"DELETE FROM UserVotes WHERE UserID = {user_id} AND PostID = {post_id};"
            cursor.execute(delete_query)
            cnx.commit()

            # Next, update the upvotes and downvotes in the posts table
            update_query = f"UPDATE Post SET Downvotes = Downvotes - 1 WHERE PostID = {post_id}"
            cursor.execute(update_query)
            cnx.commit()
            return False  # User has downvoted after the previous upvote
        else:
            return True
    else:
        return False  # User has not upvoted the post, they can upvote it


# How to get a User's OrgID

In [11]:
query=f"SELECT OrganizationID from User where UserID=1"
cursor.execute(query)
result=cursor.fetchall()
print(result)

[(None,)]


# Analytics time

In [1]:
import matplotlib.pyplot as plt


In [35]:
def get_sentiment_analytics(post_id):
    q=f"SELECT * from Comment WHERE PostID={post_id};"
    cursor.execute(q)
    result=cursor.fetchall()
    comment_df=pd.DataFrame(result,columns=["CommentID","Content","CreatedAtDate","UserID","PostID"])
    comments=comment_df["Content"]
    sentiments=[]
    for comment in comments:
        sentiment_scores = sid.polarity_scores(comment)
        compound_score = sentiment_scores['compound']
        sentiments.append(compound_score)
    comment_df['CompoundScore'] = sentiments

    # Categorize comments as positive, negative, or neutral
    comment_df['SentimentLabel'] = 'neutral'
    comment_df.loc[comment_df['CompoundScore'] > 0.05, 'SentimentLabel'] = 'positive'
    comment_df.loc[comment_df['CompoundScore'] < -0.05, 'SentimentLabel'] = 'negative'
    return comment_df


In [48]:
senti_anal=get_sentiment_analytics(6)

In [50]:
senti_anal

,CommentID,Content,CreatedAtDate,UserID,PostID,CompoundScore,SentimentLabel
0,10,I approve of this message,2023-11-04 09:59:09,NaN,6,0.0000,neutral
1,11,"this is very offensive, so sad to see the gene...",2023-11-04 09:59:40,NaN,6,-0.7281,negative
2,12,Very good,2023-11-04 09:59:52,NaN,6,0.4927,positive
3,14,very poor quality post,2023-11-04 10:03:46,8.0,6,-0.5256,negative


In [49]:
senti_anal['SentimentLabel'].value_counts().index

Index(['negative', 'neutral', 'positive'], dtype='object')

In [45]:
senti_anal['SentimentLabel'].value_counts().values

array([4, 1], dtype=int64)

In [44]:
grouped = senti_anal.groupby(['CreatedAtDate', 'SentimentLabel']).size().unstack().fillna(0)
grouped['neutral'].values,grouped.index

(array([2., 1., 1., 0.]),
 DatetimeIndex(['2023-11-02 09:26:24', '2023-11-04 08:32:18',
                '2023-11-04 08:51:24', '2023-11-04 09:02:50'],
               dtype='datetime64[ns]', name='CreatedAtDate', freq=None))

In [46]:
def fetch_comments_for_post(post_id):
    query = "SELECT Comment.CommentID, Comment.Content, Comment.CreatedAtDate, User.Name " \
            "FROM Comment " \
            "INNER JOIN User ON Comment.UserID = User.UserID " \
            "WHERE Comment.PostID = %s"
    cursor.execute(query, (post_id,))
    comments = []
    
    for comment_data in cursor.fetchall():
        comment = {
            'CommentID': comment_data[0],
            'Content': comment_data[1],
            'CreatedAtDate': comment_data[2],
            'Name': comment_data[3]
        }
        comments.append(comment)

    return comments